In [1]:
import notebook_loader
from model import multiple_layer_model as mlp

importing notebook from /data1/dhyoon/tensor_work/book_main/model/multiple_layer_model.ipynb


In [2]:
mlp.random_fix = True

In [3]:
id1 = mlp.IrisDataset("regression")

m1 = mlp.MultiLayerPerceptronModel("iris-regression-test-hid-1", id1, [10])
m1.train(epoch_count=100)

m2 = mlp.NoTFMLPModel("notf-iris-regression-hid-1", id1, [10])
m2.train(epoch_count=100)

Model iris-regression-test-hid-1 train report:
    Epoch 10: cost=0.548, accuracy=0.070/-0.261 (0/0 secs)
    Epoch 20: cost=0.264, accuracy=-0.211/-0.654 (0/0 secs)
    Epoch 30: cost=0.229, accuracy=-0.134/-0.556 (1/1 secs)
    Epoch 40: cost=0.196, accuracy=-0.028/-0.441 (0/1 secs)
    Epoch 50: cost=0.165, accuracy=0.067/-0.314 (0/1 secs)
    Epoch 60: cost=0.137, accuracy=0.169/-0.178 (1/2 secs)
    Epoch 70: cost=0.112, accuracy=0.280/-0.052 (0/2 secs)
    Epoch 80: cost=0.092, accuracy=0.379/0.078 (0/2 secs)
    Epoch 90: cost=0.077, accuracy=0.472/0.194 (0/2 secs)
    Epoch 100: cost=0.066, accuracy=0.547/0.297 (1/3 secs)

Model notf-iris-regression-hid-1 train report:
    Epoch 10: cost=0.548, accuracy=-0.228/-0.642 (0/0 secs)
    Epoch 20: cost=0.264, accuracy=0.169/-0.162 (0/0 secs)
    Epoch 30: cost=0.229, accuracy=0.175/-0.090 (0/0 secs)
    Epoch 40: cost=0.196, accuracy=0.266/-0.014 (0/0 secs)
    Epoch 50: cost=0.165, accuracy=0.317/0.059 (0/0 secs)
    Epoch 60: cost=

In [4]:
import numpy as np
import tensorflow as tf
import time

In [5]:
class SudokuDataset(mlp.Dataset):
    pass

def sudoku_init(self, max_cnt=0, train_ratio=0.8, valid_ratio=0.05):
    mlp.Dataset.__init__(self, "sudoku")
    
    if mlp.random_fix: np.random.seed(3456)
    
    xs, ys = load_sudoku_file("./data/sudoku.csv", max_cnt)

    data_count = len(xs)
    tr_count = int(data_count * train_ratio)
    va_count = int(data_count * valid_ratio)
    te_count = data_count - tr_count - va_count
    print("sudoku data was prepared {}+{}+{}={} items". \
         format(tr_count, va_count, te_count, data_count))
    te_start = tr_count + va_count

    indices = np.arange(data_count)
    np.random.shuffle(indices)

    self.train_xs = xs[indices[0:tr_count]]
    self.train_ys = ys[indices[0:tr_count]]
    self.validate_xs = xs[indices[tr_count:te_start]]
    self.validate_ys = ys[indices[tr_count:te_start]]
    self.test_xs = xs[indices[te_start:]]
    self.test_ys = ys[indices[te_start:]]

    self.input_dim = 81
    self.output_dim = 81 * 9
    self.train_count = tr_count
    
SudokuDataset.__init__ = sudoku_init

In [6]:
def load_sudoku_file(filepath, max_cnt):
    quizzes, solutions = [], []

    for line in open(filepath):
        if line[0] == 'q': continue
        quiz, solution = unpack_map(line)
        quizzes.append(quiz)
        solutions.append(solution)
        if max_cnt > 0 and len(quizzes) >= max_cnt: break

    xs = np.asarray(quizzes)
    ones = np.ones([81]).astype(int)
    solution_idxs = solutions - ones
    ys = np.eye(9)[solution_idxs].reshape(-1, 81*9)

    return xs, ys
    
def unpack_map(line):
    quiz = np.zeros([81], dtype=np.int8)
    solution = np.zeros([81], dtype=np.int8)
    for n in range(81):
        quiz[n] = int(line[n])
        solution[n] = int(line[n+82])
    return quiz, solution

In [7]:
class SudokuMlpModel(mlp.MultiLayerPerceptronModel):
    pass

def sudoku_mlp_init(self, name, dset, hdims, learning_rate=0.01):
    mlp.MultiLayerPerceptronModel.__init__(self, name, \
                         dset, hdims, learning_rate)

SudokuMlpModel.__init__ = sudoku_mlp_init

In [20]:
def sudoku_train(self, epoch_count=10, batch_size=10):
    if batch_size == 0:
        batch_size = self.dataset.train_count
        
    batch_count = int(self.dataset.train_count / batch_size)
    report_period = epoch_count / 10
    
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    if mlp.random_fix: np.random.seed(1945)
    
    time1 = time2 = int(time.time())
    
    print("Model {} train report:".format(self.name))
    
    dset = self.dataset
    run_targets = [self.train_op, self.loss, \
                   self.total_acc, self.blank_acc, self.max_acc]
    acc_targets = [self.total_acc, self.blank_acc, self.max_acc]

    for epoch in range(epoch_count):
        costs, total_accs, blank_accs, max_accs =  [], [], [], []
        
        for n in range(batch_count):
            tr_X, tr_Y = dset.get_train_data(batch_size, n)
            _, cost, total_acc, blank_acc, max_acc = \
                sess.run(run_targets, \
                         feed_dict={self.x:tr_X, self.y:tr_Y})
            costs.append(cost)
            total_accs.append(total_acc)
            blank_accs.append(blank_acc)
            max_accs.append(max_acc)
        
        if (epoch+1) % report_period == 0:
            va_X, va_Y = dset.get_test_data(True, 30)
            total_acc, blank_acc, max_acc = \
                sess.run(acc_targets, \
                         feed_dict={self.x:va_X, self.y:va_Y})

            time3 = int(time.time())
            print("    Epoch {}: cost={:5.3f} ({}/{} secs) \n \
    accuracy={:5.3f}/{:5.3f}/{:5.3f}({:5.3f}/{:5.3f}/{:5.3f})". \
            format(epoch+1,np.mean(costs),time3-time2,time3-time1, \
                   np.mean(total_accs), np.mean(blank_accs), \
                   np.mean(max_accs),total_acc,blank_acc,max_acc))
            time2 = time3

    path = "params/{}.ckpt".format(self.name)
    self.saver.save(sess, path)
    sess.close()

SudokuMlpModel.train = sudoku_train

In [9]:
def sudoku_demonstrate(self, cnt=1, detail=False):
    sess = tf.Session()
    path = "params/{}.ckpt".format(self.name)
    self.saver.restore(sess, path)
    
    print("Model {} Demonstration".format(self.name))

    total_blank_count = 0
    total_corr_count = 0
    
    for n in range(cnt):
        demo_X, demo_Y = self.dataset.get_test_data(False, 1)

        blank_count = 81 - np.count_nonzero(demo_X[0])
        wrong_count = 0

        if detail:
            y_n = np.argmax(np.reshape(demo_Y[0], [-1,9]), axis=1)
            print("x", np.reshape(demo_X[0], [-1, 9, 9]))
            print("y", np.reshape(y_n+1, [-1, 9, 9]))

        for k in range(blank_count):
            est, ans, pos = sess.run([self.max_est, self.max_ans, \
                self.max_pos], \
                       feed_dict={self.x:demo_X, self.y:demo_Y})

            row, col = (pos[0]//9)+1, (pos[0]%9)+1
            estimate, answer = int(est[0])+1, int(ans[0])+1
            msg = "correct"

            if answer != estimate:
                msg = "wrong"
                wrong_count += 1

            if detail:
                print("{}: Guess [{},{}] as {} (answer: {}) : {}". \
                     format(k+1, row, col, estimate, answer, msg))

            demo_X[0][pos] = answer + 1
   
        corr_count = blank_count - wrong_count
        corr_ratio = corr_count / blank_count
        
        if detail:
            print("{} correct guesses among {} guesses => {:5.5f}".\
                format(corr_count, blank_count, corr_ratio))

        total_blank_count += blank_count
        total_corr_count += corr_count

    corr_ratio = total_corr_count / total_blank_count
    
    print("Total: {} correct guess among {} guesses: {:5.5f}". \
           format(total_corr_count, total_blank_count, corr_ratio))
    
    sess.close()

SudokuMlpModel.demonstrate = sudoku_demonstrate

In [10]:
def sudoku_build_loss_accuracy(self):
    labels = tf.cast(tf.reshape(self.y, [-1, 81, 9]), "float")
    logits = tf.reshape(self.output, [-1, 81, 9])
    entropy = tf.nn.softmax_cross_entropy_with_logits( \
                   labels=labels, logits=logits)
    loss = tf.reduce_mean(entropy)
    
    estimate = tf.argmax(logits, 2)
    answer = tf.argmax(labels, 2)
    correct = tf.cast(tf.equal(estimate, answer), "float")
    total_acc = tf.reduce_mean(correct)
    
    bl_mask = tf.cast(1-tf.sign(self.x), "float")
    bl_correct = bl_mask * correct
    blank_acc = tf.reduce_sum(bl_correct) / tf.reduce_sum(bl_mask)

    probs = tf.nn.softmax(logits)
    max_probs = tf.reduce_max(probs, axis=2)
    bl_probs = bl_mask * max_probs
    max_pos = tf.argmax(bl_probs, 1)
    max_mask = tf.one_hot(max_pos, 81)
    max_est = tf.reduce_sum(tf.cast(estimate, "float")*max_mask, 1)
    max_ans = tf.reduce_sum(tf.cast(answer, "float")*max_mask, 1)
    max_correct = tf.cast(tf.equal(max_est, max_ans), "float")
    max_acc = tf.reduce_mean(max_correct)

    self.loss = loss
    self.accuracy = total_acc
    self.total_acc = total_acc
    self.blank_acc = blank_acc
    self.max_acc = max_acc
    self.max_est = max_est
    self.max_ans = max_ans
    self.max_pos = max_pos

SudokuMlpModel.build_loss_accuracy = sudoku_build_loss_accuracy

In [21]:
sd1 = SudokuDataset(max_cnt=1000)

sm1 = SudokuMlpModel("sudoku-mlp-1", sd1, [256])
sm1.train(epoch_count=100)

sudoku data was prepared 800+50+150=1000 items
Model sudoku-mlp-1 train report:
    Epoch 10: cost=2.199 (2/2 secs) 
     accuracy=0.123/0.123/0.139(0.110/0.108/0.167)
    Epoch 20: cost=2.187 (2/4 secs) 
     accuracy=0.138/0.137/0.170(0.118/0.116/0.100)
    Epoch 30: cost=2.177 (2/6 secs) 
     accuracy=0.152/0.150/0.194(0.123/0.122/0.167)
    Epoch 40: cost=2.166 (2/8 secs) 
     accuracy=0.165/0.160/0.226(0.118/0.114/0.167)
    Epoch 50: cost=2.156 (2/10 secs) 
     accuracy=0.177/0.172/0.260(0.135/0.133/0.167)
    Epoch 60: cost=2.144 (2/12 secs) 
     accuracy=0.187/0.181/0.295(0.135/0.126/0.133)
    Epoch 70: cost=2.132 (2/14 secs) 
     accuracy=0.197/0.190/0.317(0.127/0.117/0.100)
    Epoch 80: cost=2.119 (2/16 secs) 
     accuracy=0.207/0.198/0.321(0.128/0.113/0.067)
    Epoch 90: cost=2.105 (2/18 secs) 
     accuracy=0.218/0.209/0.341(0.126/0.109/0.000)
    Epoch 100: cost=2.090 (2/20 secs) 
     accuracy=0.229/0.218/0.375(0.130/0.113/0.067)


In [12]:
sm1.demonstrate(2, True)

INFO:tensorflow:Restoring parameters from params/sudoku-mlp-1.ckpt
Model sudoku-mlp-1 Demonstration
x [[[0 0 3 8 5 0 0 0 4]
  [1 0 0 0 0 0 6 0 0]
  [0 7 8 0 0 9 0 3 0]
  [2 0 4 3 0 0 0 0 5]
  [0 0 0 0 0 0 8 2 0]
  [0 6 9 2 0 0 1 0 3]
  [0 0 1 0 9 2 0 0 0]
  [5 8 0 0 0 1 0 0 7]
  [4 0 2 5 8 0 3 0 0]]]
y [[[9 2 3 8 5 6 7 1 4]
  [1 4 5 7 2 3 6 8 9]
  [6 7 8 1 4 9 5 3 2]
  [2 1 4 3 6 8 9 7 5]
  [3 5 7 9 1 4 8 2 6]
  [8 6 9 2 7 5 1 4 3]
  [7 3 1 6 9 2 4 5 8]
  [5 8 6 4 3 1 2 9 7]
  [4 9 2 5 8 7 3 6 1]]]
1: Guess [9,9] as 9 (answer: 1) : wrong
2: Guess [5,4] as 7 (answer: 9) : wrong
3: Guess [2,9] as 2 (answer: 9) : wrong
4: Guess [1,2] as 9 (answer: 2) : wrong
5: Guess [7,1] as 3 (answer: 7) : wrong
6: Guess [8,8] as 8 (answer: 9) : wrong
7: Guess [3,5] as 9 (answer: 4) : wrong
8: Guess [9,6] as 6 (answer: 7) : wrong
9: Guess [5,5] as 9 (answer: 1) : wrong
10: Guess [3,7] as 8 (answer: 5) : wrong
11: Guess [9,2] as 8 (answer: 9) : wrong
12: Guess [5,1] as 3 (answer: 3) : correct
13: Guess [

In [13]:
sd2 = SudokuDataset(max_cnt=10000)

sm2 = SudokuMlpModel("sudoku-mlp-2", sd2, [256])
sm2.train(epoch_count=10)

sudoku data was prepared 8000+500+1500=10000 items
Model sudoku-mlp-2 train report:
    Epoch 1: cost=2.219 (2/2 secs) 
     accuracy=0.112/0.112/0.109(0.117/0.125/0.167)
    Epoch 2: cost=2.204 (2/4 secs) 
     accuracy=0.114/0.113/0.115(0.106/0.104/0.100)
    Epoch 3: cost=2.200 (2/6 secs) 
     accuracy=0.116/0.114/0.122(0.116/0.113/0.067)
    Epoch 4: cost=2.198 (2/8 secs) 
     accuracy=0.117/0.115/0.124(0.115/0.113/0.133)
    Epoch 5: cost=2.197 (2/10 secs) 
     accuracy=0.118/0.116/0.129(0.131/0.123/0.133)
    Epoch 6: cost=2.197 (1/11 secs) 
     accuracy=0.120/0.117/0.128(0.117/0.117/0.167)
    Epoch 7: cost=2.196 (2/13 secs) 
     accuracy=0.121/0.118/0.130(0.120/0.117/0.100)
    Epoch 8: cost=2.195 (2/15 secs) 
     accuracy=0.123/0.119/0.132(0.115/0.112/0.133)
    Epoch 9: cost=2.195 (2/17 secs) 
     accuracy=0.124/0.120/0.133(0.118/0.099/0.100)
    Epoch 10: cost=2.194 (2/19 secs) 
     accuracy=0.125/0.121/0.132(0.114/0.109/0.033)


In [14]:
sm2.test()
sm2.demonstrate(2, False)

INFO:tensorflow:Restoring parameters from params/sudoku-mlp-2.ckpt
Model sudoku-mlp-2 test report: accuracy = 0.117, (0 secs)

INFO:tensorflow:Restoring parameters from params/sudoku-mlp-2.ckpt
Model sudoku-mlp-2 Demonstration
Total: 8 correct guess among 96 guesses: 0.08333


In [15]:
sm3 = SudokuMlpModel("sudoku-mlp-3", sd1, [256])
sm3.train(epoch_count=1000)

Model sudoku-mlp-3 train report:
    Epoch 100: cost=2.090 (20/20 secs) 
     accuracy=0.229/0.218/0.380(0.122/0.110/0.100)
    Epoch 200: cost=1.900 (19/39 secs) 
     accuracy=0.327/0.309/0.611(0.138/0.125/0.067)
    Epoch 300: cost=1.719 (19/58 secs) 
     accuracy=0.400/0.377/0.772(0.143/0.128/0.067)
    Epoch 400: cost=1.571 (20/78 secs) 
     accuracy=0.457/0.436/0.859(0.149/0.126/0.233)
    Epoch 500: cost=1.436 (19/97 secs) 
     accuracy=0.512/0.492/0.911(0.140/0.125/0.067)
    Epoch 600: cost=1.299 (20/117 secs) 
     accuracy=0.572/0.554/0.957(0.137/0.123/0.067)
    Epoch 700: cost=1.158 (19/136 secs) 
     accuracy=0.635/0.618/0.980(0.137/0.117/0.033)
    Epoch 800: cost=1.019 (20/156 secs) 
     accuracy=0.697/0.680/0.991(0.137/0.123/0.133)
    Epoch 900: cost=0.886 (19/175 secs) 
     accuracy=0.754/0.738/0.996(0.140/0.128/0.100)
    Epoch 1000: cost=0.764 (20/195 secs) 
     accuracy=0.808/0.796/1.000(0.137/0.124/0.100)


In [16]:
sm4 = SudokuMlpModel("sudoku-mlp-4", sd2, [256])
sm4.train(epoch_count=1000)

Model sudoku-mlp-4 train report:
    Epoch 100: cost=2.065 (194/194 secs) 
     accuracy=0.206/0.179/0.311(0.156/0.134/0.167)
    Epoch 200: cost=2.017 (193/387 secs) 
     accuracy=0.233/0.204/0.377(0.160/0.141/0.200)
    Epoch 300: cost=1.974 (190/577 secs) 
     accuracy=0.258/0.228/0.430(0.163/0.136/0.233)
    Epoch 400: cost=1.938 (191/768 secs) 
     accuracy=0.277/0.247/0.466(0.134/0.114/0.200)
    Epoch 500: cost=1.915 (192/960 secs) 
     accuracy=0.289/0.258/0.489(0.149/0.113/0.133)
    Epoch 600: cost=1.899 (193/1153 secs) 
     accuracy=0.296/0.266/0.514(0.138/0.117/0.133)
    Epoch 700: cost=1.887 (193/1346 secs) 
     accuracy=0.301/0.270/0.523(0.146/0.124/0.200)
    Epoch 800: cost=1.878 (192/1538 secs) 
     accuracy=0.306/0.274/0.533(0.153/0.131/0.067)
    Epoch 900: cost=1.870 (194/1732 secs) 
     accuracy=0.310/0.277/0.538(0.152/0.129/0.133)
    Epoch 1000: cost=1.862 (194/1926 secs) 
     accuracy=0.313/0.279/0.546(0.162/0.129/0.167)


In [17]:
sm5 = SudokuMlpModel("sudoku-mlp-5", sd2, [512])
sm5.train(epoch_count=1000)

Model sudoku-mlp-5 train report:
    Epoch 100: cost=2.031 (202/202 secs) 
     accuracy=0.226/0.198/0.363(0.158/0.129/0.133)
    Epoch 200: cost=1.947 (203/405 secs) 
     accuracy=0.274/0.244/0.474(0.152/0.135/0.100)
    Epoch 300: cost=1.848 (203/608 secs) 
     accuracy=0.323/0.292/0.577(0.152/0.116/0.200)
    Epoch 400: cost=1.766 (204/812 secs) 
     accuracy=0.359/0.327/0.656(0.153/0.126/0.167)
    Epoch 500: cost=1.711 (205/1017 secs) 
     accuracy=0.382/0.349/0.704(0.138/0.116/0.133)
    Epoch 600: cost=1.675 (205/1222 secs) 
     accuracy=0.397/0.364/0.732(0.151/0.131/0.167)
    Epoch 700: cost=1.650 (203/1425 secs) 
     accuracy=0.406/0.373/0.747(0.138/0.110/0.100)
    Epoch 800: cost=1.633 (205/1630 secs) 
     accuracy=0.413/0.379/0.758(0.142/0.127/0.100)
    Epoch 900: cost=1.619 (204/1834 secs) 
     accuracy=0.418/0.384/0.768(0.153/0.133/0.233)
    Epoch 1000: cost=1.607 (204/2038 secs) 
     accuracy=0.423/0.388/0.775(0.147/0.127/0.200)
